In [1]:
# show more than one output in cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from torchvision import models as tvm
import pretrainedmodels as ptm
models = (
    tvm.resnet101, 
    
    ptm.inceptionresnetv2,
    ptm.inceptionv4
)

DATA_DIR = 'data/4_class_11'

domain = [
#     {'name': 'model_num', 'type': 'discrete', 'domain': range(len(models))},
    {'name': 'batch_size', 'type': 'discrete', 'domain': (1, 4, 16, 24, 32, 48, 64)},
    {'name': 'adam_lr', 'type': 'discrete', 'domain': (0.001, 0.005, 0.01, 0.05, 0.1, 0.2)},
    {'name': 'adam_beta1', 'type': 'continuous', 'domain': (0.8, .99)},
    {'name': 'adam_beta2', 'type': 'continuous', 'domain': (0.95, .9999)},
    {'name': 'adam_wtdecay', 'type': 'continuous', 'domain': (0, 1)},
    {'name': 'epochs', 'type': 'discrete', 'domain': (10, 20, 30, 40, 50)}
]
#default_input = [3, 32, 0.001, 0.9, 0.999, 0, 30] # TODO: have to figure out how to do this

In [3]:
model_num = 0

def f(x):
    """ Value function to minimize for bayesian optimization """
    val_acc = train(
        model_num,
        batch_size=int(x[:,0]),
        adam_lr=float(x[:,1]),
        adam_b1=float(x[:,2]),
        adam_b2=float(x[:,3]),
        adam_wtdecay=float(x[:,4]),
        epochs=int(x[:,5])
    )
    
    return -val_acc


NUM_CLASSES = 4


def prepare_model(model_num):
    model_fn = models[model_num]
    if model_fn is tvm.resnet101:
        # torchvision models
        pretrained = True
        last = 'fc'
        image_size = 224
    else:
        # pretrainedmodels package specific differences
        pretrained = 'imagenet'
        last = 'last_linear'
        image_size = 299
    
    model = model_fn(pretrained=pretrained)
    num_in = getattr(model, last).in_features
    setattr(model, last, torch.nn.Linear(num_in, NUM_CLASSES))
    return model, image_size


import torch
from torch.optim import Adam
from torch.nn import  CrossEntropyLoss
from src.trainable import Trainable
from src.utils import get_datasets_and_loaders

iteration = 0
def train(model_num, batch_size, adam_lr, adam_b1, adam_b2, adam_wtdecay, epochs):
    global iteration
    iteration += 1
    print("\nITERATION", iteration)
    print(f'model: {models[model_num]}, batch_size: {batch_size}, epochs: {epochs}')
    print(f'adam_lr: {adam_lr}, adam_b1: {adam_b1}, adam_b2: {adam_b2}, adam_wtdecay: {adam_wtdecay}')

    model, image_size = prepare_model(model_num)
    
    _, dataloaders = get_datasets_and_loaders(DATA_DIR, 'train', 'val', 
                                              image_size=image_size)
    
    optimizer = Adam(model.parameters(), adam_lr, (adam_b1, adam_b2), adam_wtdecay)
    criterion = CrossEntropyLoss()
    
    trainable = Trainable(model, criterion, optimizer)
    
    params = '_'.join([batch_size] + [f'{param:4f}' for param in (adam_lr, adam_b1, adam_b2, adam_wtdecay)] + [epochs])
    results_fp = f'model_{model_num}/{DATA_DIR}/{params}'
    try:
        val_acc = trainable.train(dataloaders, epochs, early_stop=5, results_filepath=results_fp)
    except Exception as e:
        print(e)
        val_acc = 0
    return val_acc
    

In [4]:
# # TEST CODE
# train(model, 4, 1e-3, 0.9, 0.999, 0, 50)

## Do BO on all models on both datasets. Git push at each stage

In [5]:
from GPyOpt.methods import BayesianOptimization
import subprocess

for i in range(len(models)):
    model_num = i
    for data_index, datadir in enumerate(('data/die_vs_all_11', 'data/4_class_11')):
        global iteration
        iteration = 0
        
        DATA_DIR = datadir
        print(datadir.upper())
        NUM_CLASSES = 2 * (data_index+1)
    
        problem = BayesianOptimization(
            f=f,
            domain=domain
        )
        problem.run_optimization(max_iter=10)
        problem.plot_acquisition()
        problem.plot_convergence()
        
        # commit & push only if we can connect to internet
        command = f'git add -u && git add *.txt && commit -m "BO Results model_num={i} data={datadir} " && git push'
        process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
        output, error

DATA/DIE_VS_ALL_11

ITERATION 1
model: <function resnet101 at 0x7f23fd2c2730>, batch_size: 16, epochs: 50
adam_lr: 0.001, adam_b1: 0.8773120541969771, adam_b2: 0.9530892552080555, adam_wtdecay: 0.9218775636222142


Train:   0%|          | 0/988 [00:00<?, ?images/s]

Epoch 1/50
----------


Val:   0%|          | 0/212 [00:00<?, ?images/s]            

Train Loss: 0.5087, Acc: 0.7621


...

Train:  32%|███▏      | 316/988 [00:08<00:16, 41.21images/s]

Stopping...
Stopping...
Stopping...
Stopping...


Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f23d3def1d0>>
Traceback (most recent call last):
  File "/home/alwood/anaconda3/envs/estrous-ai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/home/alwood/anaconda3/envs/estrous-ai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/alwood/anaconda3/envs/estrous-ai/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/alwood/anaconda3/envs/estrous-ai/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/home/alwood/anaconda3/envs/estrous-ai/lib/python3.6/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/home/alwood/anaconda3/e

Stopping...


SystemExit: 1

In [ ]:
command = "spd-say 'Your code has finished, Your code has finished'"
process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)